In [41]:
from langgraph.graph import StateGraph,  START , END
from typing import TypedDict
from dotenv import load_dotenv

In [42]:
load_dotenv()

True

In [43]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)

In [44]:
model = llm

In [45]:
class email(TypedDict):
    email:str
    output:str


In [46]:
def emaildetect(state:email)-> email:
    input = state['email']

    prompt = f"${input} take this email and anylize this email as spam or ham"

    output = model.invoke(prompt).content

    state['output'] = output

    return state

In [47]:
graph = StateGraph(email)

graph.add_node('emaildetect',emaildetect)

graph.add_edge(START,'emaildetect')
graph.add_edge('emaildetect',END)

workflow = graph.compile()


In [53]:
message = {
    'email':"""Dear Valued Customer,

CRITICAL ALERT: Unusual activity detected on your account ending in ****1234!

Your BANK ACCOUNT has been flagged for suspicious login attempts from an unknown device. To prevent FRAUD and permanent LOCKOUT, you MUST verify your identity IMMEDIATELY.

CLICK HERE to secure your account: http://secure-bank-login.com/verify?user=yourname&token=ABC123XYZ

Failure to act within 24 HOURS will result in:
- Immediate account suspension
- Loss of all funds and access
- Reporting to authorities

This is your FINAL WARNING. Act now!

Best regards,
Bank Security Team
1-800-FAKE-NUM
www.realbank.com (fake link hidden)
""",
'output':""
}

output = workflow.invoke(message)

print(output['output'])

**Email Classification:** **Spam / Phishing (malicious)**  

---

## Why this message is identified as spam (phishing)

| Indicator | Explanation |
|-----------|-------------|
| **Subject line / opening** – “CRITICAL ALERT: Unusual activity detected on your account” | Uses urgency and fear‑inducing language, a classic tactic to compel immediate action. |
| **All‑caps and emphasis** – “MUST verify your identity IMMEDIATELY”, “FINAL WARNING” | Over‑use of capitalization and bold language is typical of phishing attempts. |
| **Threat of severe consequences** – “Immediate account suspension”, “Loss of all funds”, “Reporting to authorities” | Creates a sense of panic to bypass rational decision‑making. |
| **Suspicious link** – `http://secure-bank-login.com/verify?user=yourname&token=ABC123XYZ` | The domain does **not** match the legitimate bank’s domain (e.g., `bank.com`). The URL is a clear red‑flag. |
| **Fake contact information** – “1‑800‑FAKE‑NUM”, “www.realbank.com (fake link hidden)